In [16]:
#export
#General imports
import pandas as pd
import os
import pickle
import exporter
from datetime import datetime, timedelta


#email sending
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#logging
import logging
# Gets or creates a logger
logger = logging.getLogger(__name__)  
# set log level
logger.setLevel(logging.INFO)
# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)
# add file handler to logger
logger.addHandler(file_handler)

#StringMatch
from fuzzywuzzy import fuzz

#Google Trends & Tweeter  
from pytrends.request import TrendReq
import tweepy

#environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
# export
def f_send_mail(mail_content=None, 
                mail_to=None,
                mail_subject=None):
    if mail_to is None:
        mail_to = 'ali.alici84@gmail.com'
        
    if mail_content is None:
        mail_content = 'Just try mail sending logic. No specific content is set.'    
        
    if mail_subject is None:
        mail_subject = 'Test Mail from App'    

    logger.info('Try to send email to {x}'.format(x=mail_to))
    #mail_content = '''Hello,
    #This is a simple mail. There is only text, no attachments are there The mail is sent using Python SMTP library.
    #Thank You
    #'''
    
    #The mail addresses and password
    sender_address = 'cuneytsolmazto@gmail.com'
    sender_pass = os.environ.get('GMAIL_API_SECRET')
    receiver_address = mail_to
    
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = mail_subject   #The subject line
    #The body and the attachments for the mail
    message.attach(MIMEText(mail_content, 'plain'))
    try:
        #Create SMTP session for sending the mail
        session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
        session.starttls() #enable security
        session.login(sender_address, sender_pass) #login with mail_id and password
        text = message.as_string()
        session.sendmail(sender_address, receiver_address, text)
        session.quit()
        logger.info('Mail Sent')
    except:
        logger.error('Sent mail has failed !!')
        session.quit()

In [39]:
#export
def connect_tweet():    
    # OAuth process, using the keys and tokens
    consumer_key = os.environ.get('TWEET_CONSUMER_KEY')
    consumer_secret = os.environ.get('TWEET_CONSUMER_SECRET')
    access_token = os.environ.get('TWEET_ACCESS_TOKEN')
    access_token_secret = os.environ.get('TWEET_ACCESS_TOKEN_SECRET')
    logger.info('Try to connect tweeter api')

    try:        
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        logger.info('Successfully connected to tweeter api')
    except:
        logger.error('tweeter connection has failed !!')
        print('ERROR!!!')
        
    return api



def get_tweet_trends_by_loc(v_api, v_loc):
    if v_loc == "TR":
        woeid = 23424969
    elif v_loc == "US":
        woeid = 23424977    

    # fetching the trends
    trends = v_api.trends_place(id = woeid)
    list_tmp = []
    for value in trends:
        for trend in value['trends']:
            list_tmp.append(trend['name'])  
            #print(trend['name'])

    return list_tmp


def get_tweets_from_user(v_api, v_userid):
    # fetching the tweets
    list_tmp = []
    tweets = v_api.user_timeline(screen_name=v_userid, 
                               # 100 is the maximum allowed count
                               count=100,
                               include_rts = False,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               #tweet_mode = 'extended'
                               )


    # printing the statuses
    for info in tweets:
        dt_hours_from_now = datetime.now() + timedelta(hours=-12)
        if info.created_at >= dt_hours_from_now: 
            txt = "ID: {id}, Date: {dt}, Text: {txt}".format(id=info.id, dt= info.created_at, txt= info.text )
            list_tmp.append(txt)
            #print("ID: {}".format(info.id))
            #print(info.created_at)
            #print(info.text, end = "\n\n")

    return list_tmp



def tweet_analyze():
    api = connect_tweet()

    tweet_dict = {}
    tweet_dict.update({"TR TREND TOPICS": get_tweet_trends_by_loc(api, "TR")}) 
    tweet_dict.update({"US TREND TOPICS": get_tweet_trends_by_loc(api, "US")}) 
    
    tweetuserid = 'elonmusk'
    tweet_dict.update({"TWEET from " + tweetuserid: get_tweets_from_user(api, tweetuserid)})
    
    return tweet_dict


def match_symbol_with_tweets(v_score=90):

    file = open("coin_list_df.pkl","rb")
    coin_list_df = pickle.load(file)

    dict_tw = tweet_analyze()
    list_tmp=[]
    for index, row in coin_list_df.iterrows():
        for key in dict_tw:
            for trendtopic in dict_tw.get(key):
                name_score = fuzz.token_set_ratio(row["Name"].lower(), trendtopic.lower())
                symbol_score = fuzz.token_set_ratio(row["Symbol"].lower(), trendtopic.lower())
                if name_score > v_score | symbol_score > v_score :
                    list_tmp.append([key, row["Name"], row["Symbol"], trendtopic.lower(), name_score, symbol_score  ])

    return list_tmp



def send_mails_with_matches():
    v_list = match_symbol_with_tweets(v_score=90)
    mail_body = ''
    for topic in v_list:
        mail_body = mail_body + ' $ '.join(topic[0:4]) + '\n'  
    if len(v_list) > 0:
        f_send_mail(mail_content = mail_body)
    

In [41]:
match_symbol_with_tweets(v_score=90)

[['US TREND TOPICS', 'Monero', 'XMR', 'joe rogan', 53, 17],
 ['TR TREND TOPICS', 'Terra', 'LUNA', 'werner', 55, 20],
 ['TR TREND TOPICS', 'Terra', 'LUNA', 'şeriat', 55, 20],
 ['US TREND TOPICS', 'Terra', 'LUNA', 'werner', 55, 20],
 ['US TREND TOPICS', 'Cosmos', 'ATOM', 'colossus', 57, 17],
 ['TR TREND TOPICS', 'Maker', 'MKR', 'sackey', 55, 22],
 ['US TREND TOPICS', 'Maker', 'MKR', '#rmache', 55, 22],
 ['US TREND TOPICS', 'Tezos', 'XTZ', 'tommys', 55, 22],
 ['US TREND TOPICS', 'Algorand', 'ALGO', 'varane', 57, 20],
 ['US TREND TOPICS', 'Avalanche', 'AVAX', 'varane', 67, 40],
 ['TR TREND TOPICS', 'Kusama', 'KSM', 'kamara', 67, 44],
 ['TR TREND TOPICS', 'THORChain', 'RUNE', 'harca', 57, 22],
 ['US TREND TOPICS', 'Elrond', 'EGLD', 'on me', 55, 22],
 ['TR TREND TOPICS', 'NEM', 'XEM', 'benzema', 60, 40],
 ['US TREND TOPICS', 'NEM', 'XEM', 'benzema', 60, 40],
 ['TR TREND TOPICS', 'Stacks', 'STX', 'sackey', 67, 22],
 ['US TREND TOPICS', 'The Graph', 'GRT', 'the cdc', 60, 20],
 ['US TREND TOPIC

In [53]:
def send_mails_with_matches():
    v_list = match_symbol_with_tweets(v_score=90)
    mail_body = ''
    for topic in v_list:
        mail_body = mail_body + ' $ '.join(topic[0:4]) + '\n'  
    if len(v_list) > 0:
        f_send_mail(mail_content = mail_body)
    

In [51]:
# export
if __name__ == "__main__":
    v_list = match_symbol_with_tweets(v_score=90)
    mail_body = ''
    for topic in v_list:
        mail_body = mail_body + ' $ '.join(topic[0:4]) + '\n'  
    if len(v_list) > 0:
        f_send_mail(mail_content = mail_body)

In [9]:
###TRIAL AREA
api = connect_tweet()

userID = 'elonmusk'

#new_tweets = api.user_timeline(screen_name=screen_name, count=200)

tweets = api.user_timeline(screen_name=userID, 
                           # 100 is the maximum allowed count
                           count=100,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           #tweet_mode = 'extended'
                           )


# printing the statuses
for info in tweets:
    print("ID: {}".format(info.id))
    print(info.created_at)
    print(info.text, end = "\n\n")





ID: 1387140919145766912
2021-04-27 20:25:41
@PPathole Yup

ID: 1387140695387951105
2021-04-27 20:24:48
@garyblack00 @CGrantWSJ @WSJ I find it more helpful to read/follow individual journalists, rather than publications… https://t.co/zDRXthEyDc

ID: 1386841861323698177
2021-04-27 00:37:20
@kchangnyt https://t.co/eLvYOnn6zT

ID: 1386836238771105793
2021-04-27 00:15:00
Starship SN15 static fire completed, preparing for flight later this week

ID: 1386835900169129984
2021-04-27 00:13:39
@jeff_foust True 🤣🤣

ID: 1386825367948644352
2021-04-26 23:31:48
@kchangnyt Can’t get it up (to orbit) lol

ID: 1386821871199571968
2021-04-26 23:17:54
@heydave7 Seems quite likely at least based on revenue in 2022 &amp; possibly total units in 2023

ID: 1386821144037236737
2021-04-26 23:15:01
@stoolpresidente No, you do not. I have not sold any of my Bitcoin. Tesla sold 10% of its holdings essentially to p… https://t.co/JMnjABou4g

ID: 1386816432655290381
2021-04-26 22:56:18
@jgrano305 @Orangetilt Yes

ID:

In [27]:
tweets

[Status(_api=<tweepy.api.API object at 0x7f43a9f51fd0>, _json={'created_at': 'Tue Apr 27 00:37:20 +0000 2021', 'id': 1386841861323698177, 'id_str': '1386841861323698177', 'text': '@kchangnyt https://t.co/eLvYOnn6zT', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'kchangnyt', 'name': 'Kenneth Chang', 'id': 2463358249, 'id_str': '2463358249', 'indices': [0, 10]}], 'urls': [], 'media': [{'id': 1386841857192337420, 'id_str': '1386841857192337420', 'indices': [11, 34], 'media_url': 'http://pbs.twimg.com/media/Ez8NUkGXoAwsbw2.jpg', 'media_url_https': 'https://pbs.twimg.com/media/Ez8NUkGXoAwsbw2.jpg', 'url': 'https://t.co/eLvYOnn6zT', 'display_url': 'pic.twitter.com/eLvYOnn6zT', 'expanded_url': 'https://twitter.com/elonmusk/status/1386841861323698177/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 672, 'h': 1200, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 854, 'h': 1525, 'resize': 'fit'}, 'small': 

In [54]:
from exporter import export
export("GeneralFunctions.ipynb","GeneralFunctions.py")